# 1. Proof of plaquette expansion

In [ ]:
from sympy import symbols, Matrix, latex, expand, simplify
from IPython.display import display, Math

# Define non-commutative symbols
a1,a3,a5,a12,a10,a8 = symbols('\\tau_1 \\tau_3 \\tau_5 \\tau_{12} \\tau_{10} \\tau_8', commutative=False)
b2,b4,b7,b11,b9,b6 = symbols('\\sigma_2 \\sigma_4 \\sigma_7 \\sigma_{11} \\sigma_9 \\sigma_6', commutative=False)
c = symbols('\\alpha')

In [ ]:
res = expand((1+c*(a1*b2*a3))*(1+c*(a3*b4*a5))*(1+c*(a5*b7*a12))*(1+c*(a12*b11*a10))*(1+c*(a10*b9*a8))*(1+c*(a8*b6*a1)))

In [ ]:
# Define a substitution rule for Pauli matrices squaring to identity
subs_rules = {
    a1**2: 1, a3**2: 1, a5**2: 1, a12**2: 1, a10**2: 1, a8**2: 1,
    b2**2: 1, b4**2: 1, b7**2: 1, b11**2: 1, b9**2: 1, b6**2: 1
}

# Simplify the result using the substitution rule
simplified_res = res.subs(subs_rules)

# Display the simplified result
simplified_res

In [ ]:
g_r = 0.85
lamb = (1/g_r) ** (1/6) 
print(lamb)
1/g_r

In [ ]:
from IPython.display import Markdown as md

md(f"The result of the expansion is: \n ${latex(simplified_res)}$")

# 2. MPO for Z2 Dynamical Matter on HeavyHex architecture

In [ ]:
def generate_subgroups(l, site):
    if (site%2) == 0:
        # Initialize the list with the correct starting point
        subgroups = ["down"] if l % 2 == 0 else ["up"]
        # Alternate "up" and "down" to build the list
        for i in range(1, l + 1):
            subgroups.append("up" if subgroups[-1] == "down" else "down")
    elif (site%2) == 1:
        # Initialize the list with the correct starting point
        subgroups = ["up"] if l % 2 == 0 else ["down"]
        # Alternate "up" and "down" to build the list
        for i in range(1, l + 1):
            subgroups.append("down" if subgroups[-1] == "up" else "up")
    
    return subgroups

generate_subgroups(l=1,site=2)

In [ ]:
from ncon import ncon
from scipy.sparse import csc_array, identity, linalg
import numpy as np

def sparse_non_diag_paulis_indices(n: int, N: int):
    """
    Returns a tuple (row_indices, col_indices) containing the row and col indices of the non_zero elements
    of the tensor product of a non diagonal pauli matrix (x, y) acting over a single qubit in a Hilbert
    space of N qubits

    """
    if 0 <= n < N:
        block_length = 2 ** (N - n - 1)
        nblocks = 2**n
        ndiag_elements = block_length * nblocks
        k = np.arange(ndiag_elements, dtype=int)
        red_row_col_ind = (k % block_length) + 2 * (k // block_length) * block_length
        upper_diag_row_indices = red_row_col_ind
        upper_diag_col_indices = block_length + red_row_col_ind
        row_indices = np.concatenate((upper_diag_row_indices, upper_diag_col_indices))
        col_indices = np.concatenate((upper_diag_col_indices, upper_diag_row_indices))
        return row_indices, col_indices
    else:
        raise ValueError("Index n must fulfill 0 <= n < N")


def sparse_pauli_x(
    n: int,
    L: int,
    row_indices_cache: np.ndarray = None,
    col_indices_cache: np.ndarray = None,
):
    """
    Returns a CSC sparse matrix representation of the pauli_x matrix acting over qubit n in a Hilbert space of L qubits
    0 <= n < L

    """
    if 0 <= n < L:
        if (row_indices_cache is None) or (col_indices_cache is None):
            row_indices_cache, col_indices_cache = sparse_non_diag_paulis_indices(n, L)
        data = np.ones_like(row_indices_cache)
        result = csc_array(
            (data, (row_indices_cache, col_indices_cache)), shape=(2**L, 2**L)
        )  # , dtype=complex
        return result
    else:
        raise ValueError("Index n must fulfill 0 <= n < L")
    

def sparse_pauli_z(n: int, L: int):
    """
    Returns a CSC sparse matrix representation of the pauli_z matrix acting over qubit n in a Hilbert space of L qubits
    0 <= n < L

    """
    if 0 <= n < L:
        block_length = 2 ** (L - n)
        nblocks = 2**n
        block = np.ones(block_length, dtype=int)
        block[block_length // 2 : :] = -1
        diag = np.tile(block, nblocks)
        row_col_indices = np.arange(2**L, dtype=int)
        result = csc_array(
            (diag, (row_col_indices, row_col_indices)),
            shape=(2**L, 2**L),
            dtype=complex,
        )
        return result
    else:
        raise ValueError("Index n must fulfill 0 <= n < L")


def mpo_skeleton(l: int, l_int: int = None, aux_dim: int = None):
    """
    mpo_skeleton

    This function initializes the mpo tensor or shape (2+l,2+l,2**l,2**l)
    with O matrices. We add as well the identities in the first and last
    element of the mpo tensor.

    aux_dim: int - This auxiliary dimension represents how many rows and
                columns we want in our MPO. By default None means that it adapts
                to the system under study. Fixing the auxiliary dimension is
                useful for known observables which will not need larger MPOs
    """
    if l_int == None:
        l_int = l
    I = identity(2**l_int, dtype=complex)
    O = csc_array((2**l_int, 2**l_int), dtype=complex)
    if aux_dim == None:
        skeleton = np.array(
            [[O.toarray() for i in range(2 + l)] for j in range(2 + l)]
        )
    else:
        skeleton = np.array(
            [[O.toarray() for i in range(aux_dim)] for j in range(aux_dim)]
        )
    skeleton[0, 0] = I.toarray()
    skeleton[-1, -1] = I.toarray()
    mpo = skeleton
    return mpo

def mpo_Z2_heavy_hex(L, l, J, h, lamb, U=1000):

    mpo_list = []
    for c in range(L):
        mpo = mpo_skeleton(l=l, l_int=3*(l+1), aux_dim=(l+3))

        subgroups = generate_subgroups(l=l, site=c)
        print(f"\n >> subgroups: {subgroups}\n")
        for i, subgroup in enumerate(subgroups):
            print(f"\n*** finding hamiltonian terms for subgroup: {subgroup}, number: {i}, site: {c}\n")
            # local matter term
            print("     local matter term")
            mpo[0,-1] += - J * sparse_pauli_z(n=i * 3, L=3*(l+1)).toarray()
            # local horizontal gauge term
            print("     local horizontal gauge term")
            mpo[0,-1] += - h * sparse_pauli_z(n=i * 3 + 1, L=3*(l+1)).toarray()
            # local vertical gauge term
            print("     local vertical gauge term")
            mpo[0,-1] += - h * sparse_pauli_z(n=i * 3 + 2, L=3*(l+1)).toarray()
        
            if i > 0 and subgroups[i] == "up" and subgroups[i-1] == "down":
                # subtract redundant vertical gauge term
                print("     subtract redundant vertical gauge term")
                mpo[0,-1] -= - h * sparse_pauli_z(n=i * 3 + 2, L=3*(l+1)).toarray()
                # interaction vertical term
                print("     interaction vertical term")
                mpo[0,-1] += - lamb * sparse_pauli_x(n=((i-1) * 3), L=3*(l+1)).toarray() @ sparse_pauli_x(n=((i-1) * 3) + 2, L=3*(l+1)).toarray() @ sparse_pauli_x(n=(i * 3), L=3*(l+1)).toarray()

            # start intercation horizontal term (matter + gauge)
            print("     start intercation horizontal term (matter + gauge)")
            mpo[0,i+1] = sparse_pauli_x(n=(i * 3), L=3*(l+1)).toarray() @ sparse_pauli_x(n=(i * 3) + 1, L=3*(l+1)).toarray()
            # start intercation horizontal term (matter + gauge)
            print("     end intercation horizontal term (matter)")
            mpo[i+1,-1] = - lamb * sparse_pauli_x(n=(i * 3), L=3*(l+1)).toarray()
            
            if U != 0:
                # separating physical states with gauss law penalization
                print("     gauss law penalization term")
                G = sparse_pauli_z(n=(i * 3), L=3*(l+1)) @ sparse_pauli_z(n=(i * 3) + 1, L=3*(l+1)) @ sparse_pauli_z(n=(i * 3) + 2, L=3*(l+1))
                mpo[0,-1] += U * (G.toarray() - identity(2**(3*(l+1))).toarray()) @ (G.toarray() - identity(2**(3*(l+1))).toarray())
            
            # mpo[0,(i+1)+len(subgroups)] = sparse_pauli_z(n=(i * 3) + 1, L=3*(l+1)) @ sparse_pauli_z(n=(i * 3) + 2, L=3*(l+1))
            # G = sparse_pauli_z(n=((i-1) * 3) + 1, L=3*(l+1)) @ sparse_pauli_z(n=(i * 3), L=3*(l+1)) @ sparse_pauli_z(n=(i * 3) + 1, L=3*(l+1)) @ sparse_pauli_z(n=(i * 3) + 2, L=3*(l+1))
        
        mpo_list.append(mpo)
        mpo = mpo_skeleton(l=l, l_int=3*(l+1), aux_dim=(l+3))
    return mpo_list

In [ ]:
L = 4
l = 1
mpo = mpo_Z2_heavy_hex(L, l=l, J=10, h=5, lamb=1)
mpo[0].shape, [(l+3), (2**3)**(l+1)]

## 2.1 Run the DMRG to inspect the parameter range

In [ ]:
from qs_mps.mps_class import MPS
from qs_mps.utils import tensor_shapes
chi = 30
L = 4
l = 1
phys_dim = (2**3)**(l+1)

mps = MPS(L=L, d=phys_dim, model="Z2_higgs", chi=chi)
mps._random_state(seed=3, type_shape="rectangular", chi=chi)
mps.canonical_form()
tensor_shapes(mps.sites)

mpo = mpo_Z2_heavy_hex(L, l=l, J=10, h=1, lamb=1, U=0)
tensor_shapes(mpo) 
print([(l+3), (2**3)**(l+1)])
mps.w = mpo

mps.DMRG(trunc_chi=True, trunc_tol=False, where=L//2)


### 2.1.1 Mapping of $g$ into $h$ and $\lambda$

We have a Pure (static matter) $\mathcal{Z}_2$ LGT with electric and plaquette terms and the coefficients are, respectively, $g$, and $1/g$.

We want now to use the results from the section **1.** to map the couplings in the dynamical matter Theory.

$$ g \leftrightarrow h \quad ; \quad \left(\frac{1}{g}\right)^{\frac{1}{6}} \leftrightarrow \lambda \quad .$$

In [ ]:
gs = np.logspace(-3,-1,20)
# gs = np.linspace(0.01,0.1,20)
chi = 32
L = 4
l = 1
phys_dim = (2**3)**(l+1)

# check params range
for g in gs:
    h = g
    lamb = (1/g)**(1/6)
    print(f"\n ####### params h: {h}, lambda: {lamb}\n")

In [ ]:
energies = []
entropies = []
sms = []
times = []

for g in gs:
    h = g
    lamb = (1/g)**(1/6)
    print(f"\n ####### params h: {h}, lambda: {lamb}\n")
    mps = MPS(L=L, d=phys_dim, model="Z2_higgs", chi=chi)
    mps._random_state(seed=3, type_shape="rectangular", chi=chi)
    mps.canonical_form()
    tensor_shapes(mps.sites)

    mpo = mpo_Z2_heavy_hex(L, l=l, J=10, h=h, lamb=lamb, U=0)
    tensor_shapes(mpo) 
    print([(l+3), (2**3)**(l+1)])
    mps.w = mpo

    energy, entropy, schmidt_val, time = mps.DMRG(trunc_chi=True, trunc_tol=False, where=L//2)
    energies.append(energy[-1])
    entropies.append(entropy)
    sms.append(schmidt_val)
    times.append(time)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(gs, entropies)
plt.grid(True)
plt.title("Static entropy, vacuum sector")

In [ ]:
plt.plot(gs, energies)
plt.grid(True)
plt.title("Ground state energy, vacuum sector")

In [ ]:
plt.plot(gs, np.asarray(sms)[:,0,-1])
plt.yscale('log')
plt.grid(True)
plt.title("Last schmidt values, vacuum sector")